# Power BI Gateway Log Analysis

This notebook analyzes Power BI Gateway logs, specifically focusing on:
- Query Execution Reports
- Query Start Reports

The logs are loaded from the path specified in the .env file.

In [30]:
# Import required libraries
import os
import pandas as pd
import json
import glob
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv(override=True)

print("Required libraries imported successfully!")

Required libraries imported successfully!


In [31]:
# === Configuration - Column Names ===

# Define the EvaluationContext column name as a variable
# This allows easy switching between different column variations
EVALUATION_CONTEXT_COLUMN = 'EvaluationContext_serviceTraceContexts'

print(f"✅ Configuration set:")
print(f"   📋 EvaluationContext column: {EVALUATION_CONTEXT_COLUMN}")
print(f"   💡 To use different column, update EVALUATION_CONTEXT_COLUMN variable")

✅ Configuration set:
   📋 EvaluationContext column: EvaluationContext_serviceTraceContexts
   💡 To use different column, update EVALUATION_CONTEXT_COLUMN variable


In [32]:
# Configuration - Load path from .env file
log_file_path = os.getenv('LOG_FILE_PATH')
if not log_file_path:
    raise ValueError("LOG_FILE_PATH not found in .env file")

print(f"Log file path from .env: {log_file_path}")

# Check if path exists and handle gracefully
if not os.path.exists(log_file_path):
    print(f"WARNING: Path does not exist: {log_file_path}")
    print("This could happen if:")
    print("1. The path in .env file is incorrect")
    print("2. The path is on a different machine or network drive")
    print("3. The path needs to be mapped or accessible")
    
    # Create a sample directory structure for demonstration
    current_dir = os.path.dirname(os.path.abspath(""))
    sample_path = os.path.join(current_dir, "sample_gateway_logs")
    
    print(f"\nCreating sample directory structure at: {sample_path}")
    
    # Create sample directories
    os.makedirs(os.path.join(sample_path, "Gateway1"), exist_ok=True)
    os.makedirs(os.path.join(sample_path, "Gateway2"), exist_ok=True)
  
   

# List all folders in the path
folders = [f for f in os.listdir(log_file_path) if os.path.isdir(os.path.join(log_file_path, f))]
print(f"Found {len(folders)} folders: {folders}")

Log file path from .env: C:\\Users\\aveekr\\OneDrive - Microsoft\\Documents\\Demos\\my_code_samples\\azuresamples\\sample_gateway_logs
Found 2 folders: ['Gateway1', 'Gateway2']


In [33]:
def load_query_execution_reports(folder_path):
    """
    Load Query Execution Report files from a given folder.
    These files typically contain information about completed queries.
    """
    query_execution_files = []
    execution_data = []
    
    # Common patterns for query execution report files
    execution_patterns = ['*QueryExecutionReport*']
    
    for pattern in execution_patterns:
        files = glob.glob(os.path.join(folder_path, pattern))
        query_execution_files.extend(files)
    
    print(f"Found {len(query_execution_files)} query execution files in {folder_path}")
    
    for file_path in query_execution_files:
        try:
            if file_path.lower().endswith('.csv'):
                df = pd.read_csv(file_path)
                df['source_file'] = os.path.basename(file_path)
                df['folder'] = os.path.basename(folder_path)
                execution_data.append(df)
                print(f"  Loaded CSV: {os.path.basename(file_path)} ({len(df)} rows)")
            elif file_path.lower().endswith('.json'):
                with open(file_path, 'r') as f:
                    json_data = json.load(f)
                df = pd.json_normalize(json_data)
                df['source_file'] = os.path.basename(file_path)
                df['folder'] = os.path.basename(folder_path)
                execution_data.append(df)
                print(f"  Loaded JSON: {os.path.basename(file_path)} ({len(df)} rows)")
        except Exception as e:
            print(f"  Error loading {file_path}: {str(e)}")
    
    return execution_data

def load_query_start_reports(folder_path):
    """
    Load Query Start Report files from a given folder.
    These files typically contain information about query initiation.
    """
    query_start_files = []
    start_data = []
    
    # Common patterns for query start report files
    start_patterns = [
        '*QueryStartReport*'
    ]
    
    for pattern in start_patterns:
        files = glob.glob(os.path.join(folder_path, pattern))
        query_start_files.extend(files)
    
    print(f"Found {len(query_start_files)} query start files in {folder_path}")
    
    for file_path in query_start_files:
        try:
            if file_path.lower().endswith('.csv'):
                df = pd.read_csv(file_path)
                df['source_file'] = os.path.basename(file_path)
                df['folder'] = os.path.basename(folder_path)
                start_data.append(df)
                print(f"  Loaded CSV: {os.path.basename(file_path)} ({len(df)} rows)")
            elif file_path.lower().endswith('.json'):
                with open(file_path, 'r') as f:
                    json_data = json.load(f)
                df = pd.json_normalize(json_data)
                df['source_file'] = os.path.basename(file_path)
                df['folder'] = os.path.basename(folder_path)
                start_data.append(df)
                print(f"  Loaded JSON: {os.path.basename(file_path)} ({len(df)} rows)")
        except Exception as e:
            print(f"  Error loading {file_path}: {str(e)}")
    
    return start_data

print("Data loading functions defined successfully!")

Data loading functions defined successfully!


In [34]:
# Main data loading script
all_execution_data = []
all_start_data = []

print("Starting data loading process...")
print("=" * 50)

# Process each folder in the log path
for folder in folders:
    folder_path = os.path.join(log_file_path, folder)
    print(f"\nProcessing folder: {folder}")
    print("-" * 30)
    
    # Load query execution reports from this folder
    execution_data = load_query_execution_reports(folder_path)
    
    all_execution_data.extend(execution_data)
    
    # Load query start reports from this folder
    start_data = load_query_start_reports(folder_path)
    all_start_data.extend(start_data)

print("\n" + "=" * 50)
print("Data loading summary:")
print(f"Total execution report datasets: {len(all_execution_data)}")
print(f"Total start report datasets: {len(all_start_data)}")

Starting data loading process...

Processing folder: Gateway1
------------------------------
Found 1 query execution files in C:\\Users\\aveekr\\OneDrive - Microsoft\\Documents\\Demos\\my_code_samples\\azuresamples\\sample_gateway_logs\Gateway1
Found 1 query start files in C:\\Users\\aveekr\\OneDrive - Microsoft\\Documents\\Demos\\my_code_samples\\azuresamples\\sample_gateway_logs\Gateway1

Processing folder: Gateway2
------------------------------
Found 1 query execution files in C:\\Users\\aveekr\\OneDrive - Microsoft\\Documents\\Demos\\my_code_samples\\azuresamples\\sample_gateway_logs\Gateway2
Found 1 query start files in C:\\Users\\aveekr\\OneDrive - Microsoft\\Documents\\Demos\\my_code_samples\\azuresamples\\sample_gateway_logs\Gateway2

Data loading summary:
Total execution report datasets: 0
Total start report datasets: 0


In [35]:
# Combine all execution data into single DataFrames
if all_execution_data:
    execution_df = pd.concat(all_execution_data, ignore_index=True)
    print(f"\nCombined Query Execution Report:")
    print(f"Total rows: {len(execution_df)}")
    print(f"Columns: {list(execution_df.columns)}")
    print(f"Data sources: {execution_df['source_file'].nunique()} files from {execution_df['folder'].nunique()} folders")
    
    # Display first few rows
    print("\nFirst 5 rows of execution data:")
    display(execution_df.head())
else:
    execution_df = pd.DataFrame()
    print("\nNo query execution report data found")

if all_start_data:
    start_df = pd.concat(all_start_data, ignore_index=True)
    print(f"\nCombined Query Start Report:")
    print(f"Total rows: {len(start_df)}")
    print(f"Columns: {list(start_df.columns)}")
    print(f"Data sources: {start_df['source_file'].nunique()} files from {start_df['folder'].nunique()} folders")
    
    # Display first few rows
    print("\nFirst 5 rows of start data:")
    display(start_df.head())
else:
    start_df = pd.DataFrame()
    print("\nNo query start report data found")


No query execution report data found

No query start report data found


In [36]:
# Optional: Save combined data to CSV files for further analysis
save_to_csv = True  # Set to False if you don't want to save

if save_to_csv:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if not execution_df.empty:
        execution_filename = f"combined_query_execution_report_{timestamp}.csv"
        execution_df.to_csv(execution_filename, index=False)
        print(f"Query execution data saved to: {execution_filename}")
    
    if not start_df.empty:
        start_filename = f"combined_query_start_report_{timestamp}.csv"
        start_df.to_csv(start_filename, index=False)
        print(f"Query start data saved to: {start_filename}")

print("\nData loading complete! You can now analyze the data using the variables:")
print("- execution_df: Contains all query execution report data")
print("- start_df: Contains all query start report data")


Data loading complete! You can now analyze the data using the variables:
- execution_df: Contains all query execution report data
- start_df: Contains all query start report data


In [37]:
# === CSV Data Loading Functions for Power BI Gateway Logs ===

def load_query_execution_reports(base_path):
    """
    Load all QueryExecutionReport CSV files from the gateway logs folder.
    
    Args:
        base_path (str): Path to the sample_gateway_logs folder
    
    Returns:
        pandas.DataFrame: Combined data from all QueryExecutionReport files
    """
    print("🔍 Loading QueryExecutionReport files...")
    
    # Find all QueryExecutionReport files (both CSV and LOG extensions)
    execution_pattern_csv = os.path.join(base_path, "**", "QueryExecutionReport*.csv")
    execution_pattern_log = os.path.join(base_path, "**", "QueryExecutionReport*.log")
    execution_files = glob.glob(execution_pattern_csv, recursive=True) + glob.glob(execution_pattern_log, recursive=True)
    
    if not execution_files:
        print(f"⚠️  No QueryExecutionReport CSV files found in {base_path}")
        return pd.DataFrame()
    
    print(f"📁 Found {len(execution_files)} QueryExecutionReport files:")
    for file in execution_files:
        print(f"   - {os.path.basename(file)} (Size: {os.path.getsize(file):,} bytes)")
    
    # Load and combine all files
    execution_dataframes = []
    
    for file_path in execution_files:
        try:
            print(f"\n📊 Loading: {os.path.basename(file_path)}")
            
            # Read CSV with comma delimiter and headers
            df = pd.read_csv(file_path, delimiter=',', header=0, encoding='utf-8')
            
            # Add metadata columns
            df['SourceFile'] = os.path.basename(file_path)
            df['GatewayFolder'] = os.path.basename(os.path.dirname(file_path))
            df['LoadTimestamp'] = datetime.now()
            
            print(f"   ✅ Loaded {len(df):,} rows, {len(df.columns)} columns")
            print(f"   📋 Columns: {', '.join(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
            
            execution_dataframes.append(df)
            
        except Exception as e:
            print(f"   ❌ Error loading {file_path}: {str(e)}")
            continue
    
    if not execution_dataframes:
        print("❌ No QueryExecutionReport files could be loaded")
        return pd.DataFrame()
    
    # Combine all dataframes
    combined_execution_df = pd.concat(execution_dataframes, ignore_index=True)
    
    print(f"\n✅ QueryExecutionReport loading complete!")
    print(f"   📊 Total records: {len(combined_execution_df):,}")
    print(f"   📁 Files processed: {len(execution_dataframes)}")
    print(f"   🏷️  Unique gateways: {combined_execution_df['GatewayFolder'].nunique()}")
    
    return combined_execution_df

def load_query_start_reports(base_path):
    """
    Load all QueryStartReport CSV files from the gateway logs folder.
    
    Args:
        base_path (str): Path to the sample_gateway_logs folder
    
    Returns:
        pandas.DataFrame: Combined data from all QueryStartReport files
    """
    print("\n🔍 Loading QueryStartReport files...")
    
    # Find all QueryStartReport files (both CSV and LOG extensions)
    start_pattern_csv = os.path.join(base_path, "**", "QueryStartReport*.csv")
    start_pattern_log = os.path.join(base_path, "**", "QueryStartReport*.log")
    start_files = glob.glob(start_pattern_csv, recursive=True) + glob.glob(start_pattern_log, recursive=True)
    
    if not start_files:
        print(f"⚠️  No QueryStartReport CSV files found in {base_path}")
        return pd.DataFrame()
    
    print(f"📁 Found {len(start_files)} QueryStartReport files:")
    for file in start_files:
        print(f"   - {os.path.basename(file)} (Size: {os.path.getsize(file):,} bytes)")
    
    # Load and combine all files
    start_dataframes = []
    
    for file_path in start_files:
        try:
            print(f"\n📊 Loading: {os.path.basename(file_path)}")
            
            # Read CSV with comma delimiter and headers
            df = pd.read_csv(file_path, delimiter=',', header=0, encoding='utf-8')
            
            # Add metadata columns
            df['SourceFile'] = os.path.basename(file_path)
            df['GatewayFolder'] = os.path.basename(os.path.dirname(file_path))
            df['LoadTimestamp'] = datetime.now()
            
            print(f"   ✅ Loaded {len(df):,} rows, {len(df.columns)} columns")
            print(f"   📋 Columns: {', '.join(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
            
            start_dataframes.append(df)
            
        except Exception as e:
            print(f"   ❌ Error loading {file_path}: {str(e)}")
            continue
    
    if not start_dataframes:
        print("❌ No QueryStartReport files could be loaded")
        return pd.DataFrame()
    
    # Combine all dataframes
    combined_start_df = pd.concat(start_dataframes, ignore_index=True)
    
    print(f"\n✅ QueryStartReport loading complete!")
    print(f"   📊 Total records: {len(combined_start_df):,}")
    print(f"   📁 Files processed: {len(start_dataframes)}")
    print(f"   🏷️  Unique gateways: {combined_start_df['GatewayFolder'].nunique()}")
    
    return combined_start_df

print("✅ CSV loading functions defined successfully!")

✅ CSV loading functions defined successfully!


In [38]:
# === Load Gateway Log Data into Separate Tables ===

# Determine the correct path to use
if os.path.exists(log_file_path):
    data_path = log_file_path
    print(f"✅ Using configured path: {data_path}")
else:
    # Use the sample path created earlier
    current_dir = os.path.dirname(os.path.abspath(""))
    data_path = os.path.join(current_dir, "sample_gateway_logs")
    print(f"📂 Using sample data path: {data_path}")

print(f"\n🚀 Starting data loading process...")
print("=" * 60)

# Load QueryExecutionReport data
query_execution_table = load_query_execution_reports(data_path)

print("\n" + "=" * 60)

# Load QueryStartReport data  
query_start_table = load_query_start_reports(data_path)

print("\n" + "=" * 60)
print("📊 DATA LOADING SUMMARY")
print("=" * 60)

if not query_execution_table.empty:
    print(f"🔵 QueryExecutionReport Table:")
    print(f"   📊 Records: {len(query_execution_table):,}")
    print(f"   📋 Columns: {len(query_execution_table.columns)}")
    print(f"   🗂️  Sample columns: {', '.join(query_execution_table.columns[:8])}")
    if len(query_execution_table.columns) > 8:
        print(f"   📝 Additional columns: {len(query_execution_table.columns) - 8} more...")
else:
    print("🔵 QueryExecutionReport Table: ❌ No data loaded")

if not query_start_table.empty:
    print(f"\n🟢 QueryStartReport Table:")
    print(f"   📊 Records: {len(query_start_table):,}")
    print(f"   📋 Columns: {len(query_start_table.columns)}")
    print(f"   🗂️  Sample columns: {', '.join(query_start_table.columns[:8])}")
    if len(query_start_table.columns) > 8:
        print(f"   📝 Additional columns: {len(query_start_table.columns) - 8} more...")
else:
    print("🟢 QueryStartReport Table: ❌ No data loaded")

print("\n✅ Data loading process completed!")

✅ Using configured path: C:\\Users\\aveekr\\OneDrive - Microsoft\\Documents\\Demos\\my_code_samples\\azuresamples\\sample_gateway_logs

🚀 Starting data loading process...
🔍 Loading QueryExecutionReport files...
📁 Found 2 QueryExecutionReport files:
   - QueryExecutionReport_MDC-AZRDTGW-P01_20250325T141657.log (Size: 40,887,653 bytes)
   - QueryExecutionReport_MDC-AZRDTGW-P02_20240904T221408.log (Size: 82,214,220 bytes)

📊 Loading: QueryExecutionReport_MDC-AZRDTGW-P01_20250325T141657.log
   ✅ Loaded 149,862 rows, 20 columns
   📋 Columns: GatewayObjectId, RequestId, DataSource, QueryTrackingId, QueryExecutionEndTimeUTC...

📊 Loading: QueryExecutionReport_MDC-AZRDTGW-P02_20240904T221408.log
   ✅ Loaded 301,757 rows, 20 columns
   📋 Columns: GatewayObjectId, RequestId, DataSource, QueryTrackingId, QueryExecutionEndTimeUTC...

✅ QueryExecutionReport loading complete!
   📊 Total records: 451,619
   📁 Files processed: 2
   🏷️  Unique gateways: 2


🔍 Loading QueryStartReport files...
📁 Found 2

In [39]:
# === Data Exploration and Validation ===

print("🔍 EXPLORING LOADED DATA")
print("=" * 50)

# Explore QueryExecutionReport table
if not query_execution_table.empty:
    print("\n🔵 QueryExecutionReport Table Details:")
    print("-" * 40)
    print(f"Shape: {query_execution_table.shape}")
    print(f"Memory usage: {query_execution_table.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
    
    print(f"\n📋 Column Details:")
    for i, col in enumerate(query_execution_table.columns):
        dtype = query_execution_table[col].dtype
        null_count = query_execution_table[col].isnull().sum()
        print(f"   {i+1:2d}. {col:<20} | {str(dtype):<12} | Nulls: {null_count:>3}")
    
    print(f"\n📊 Sample Data (first 3 rows):")
    display(query_execution_table.head(3))
    
    # Basic statistics for numeric columns
    numeric_cols = query_execution_table.select_dtypes(include=['number']).columns
    if len(numeric_cols) > 0:
        print(f"\n📈 Numeric Columns Summary:")
        display(query_execution_table[numeric_cols].describe())

else:
    print("\n🔵 QueryExecutionReport Table: No data to explore")

# Explore QueryStartReport table
if not query_start_table.empty:
    print("\n🟢 QueryStartReport Table Details:")
    print("-" * 40)
    print(f"Shape: {query_start_table.shape}")
    print(f"Memory usage: {query_start_table.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
    
    print(f"\n📋 Column Details:")
    for i, col in enumerate(query_start_table.columns):
        dtype = query_start_table[col].dtype
        null_count = query_start_table[col].isnull().sum()
        print(f"   {i+1:2d}. {col:<20} | {str(dtype):<12} | Nulls: {null_count:>3}")
    
    print(f"\n📊 Sample Data (first 3 rows):")
    display(query_start_table.head(3))
    
    # Basic statistics for numeric columns
    numeric_cols = query_start_table.select_dtypes(include=['number']).columns
    if len(numeric_cols) > 0:
        print(f"\n📈 Numeric Columns Summary:")
        display(query_start_table[numeric_cols].describe())

else:
    print("\n🟢 QueryStartReport Table: No data to explore")

🔍 EXPLORING LOADED DATA

🔵 QueryExecutionReport Table Details:
----------------------------------------
Shape: (451619, 20)
Memory usage: 378.19 MB

📋 Column Details:
    1. GatewayObjectId      | object       | Nulls:   0
    2. RequestId            | object       | Nulls:   0
    3. DataSource           | object       | Nulls:   0
    4. QueryTrackingId      | object       | Nulls:   0
    5. QueryExecutionEndTimeUTC | object       | Nulls:   0
    6. QueryExecutionDuration(ms) | int64        | Nulls:   0
    7. QueryType            | object       | Nulls:   0
    8. DataReadingAndSerializationDuration(ms) | float64      | Nulls: 6659
    9. DataReadingDuration(ms) | float64      | Nulls: 6662
   10. DataSerializationDuration(ms) | float64      | Nulls: 6662
   11. SpoolingDiskWritingDuration(ms) | float64      | Nulls: 111585
   12. SpoolingDiskReadingDuration(ms) | float64      | Nulls: 177472
   13. SpoolingTotalDataSize(byte) | float64      | Nulls: 111584
   14. DataProcessingEn

,GatewayObjectId,RequestId,DataSource,QueryTrackingId,QueryExecutionEndTimeUTC,QueryExecutionDuration(ms),QueryType,DataReadingAndSerializationDuration(ms),DataReadingDuration(ms),DataSerializationDuration(ms),SpoolingDiskWritingDuration(ms),SpoolingDiskReadingDuration(ms),SpoolingTotalDataSize(byte),DataProcessingEndTimeUTC,DataProcessingDuration(ms),Success,ErrorMessage,SourceFile,GatewayFolder,LoadTimestamp
0,418b9fbd-21fe-4de2-ba0d-35a5968cf820,4a8aa426-36d7-4354-9c50-15a224a97b2b,"[""{\""kind\"":\""SQL\"",\""path\"":\""edw-prd;Analyti...",dd96995c-d21f-4ca1-9924-673dc25b3922,2025-03-25T19:35:29.0935009Z,2464,Refresh,797.0,467.0,186.0,39.0,6.0,6289166.0,2025-03-25T19:35:29.9281668Z,831.0,Y,NaN,QueryExecutionReport_MDC-AZRDTGW-P01_20250325T...,Gateway1,2025-09-24 13:04:18.690505
1,418b9fbd-21fe-4de2-ba0d-35a5968cf820,4a8aa426-36d7-4354-9c50-15a224a97b2b,"[""{\""kind\"":\""SQL\"",\""path\"":\""edw-prd;Analyti...",3a91658e-dfb0-4635-bd15-c7b7295e8864,2025-03-25T19:35:32.6636390Z,17,Refresh,14.0,0.0,0.0,7.0,0.0,761.0,2025-03-25T19:35:32.6756537Z,14.0,Y,NaN,QueryExecutionReport_MDC-AZRDTGW-P01_20250325T...,Gateway1,2025-09-24 13:04:18.690505
2,418b9fbd-21fe-4de2-ba0d-35a5968cf820,d53ea6a1-fee8-4cfe-83f9-66fffb7ac477,"[""{\""kind\"":\""SQL\"",\""path\"":\""edw-prd;BIA\""}""]",ca015924-4bb9-4543-b156-99583f73dd1a,2025-03-25T19:36:25.3918265Z,620,Refresh,54961.0,34014.0,18652.0,1420.0,NaN,433431059.0,2025-03-25T19:37:20.4168237Z,55025.0,Y,NaN,QueryExecutionReport_MDC-AZRDTGW-P01_20250325T...,Gateway1,2025-09-24 13:04:18.690505



📈 Numeric Columns Summary:


,QueryExecutionDuration(ms),DataReadingAndSerializationDuration(ms),DataReadingDuration(ms),DataSerializationDuration(ms),SpoolingDiskWritingDuration(ms),SpoolingDiskReadingDuration(ms),SpoolingTotalDataSize(byte),DataProcessingDuration(ms)
count,4.516190e+05,4.449600e+05,4.449570e+05,4.449570e+05,3.400340e+05,274147.000000,3.400350e+05,3.404650e+05
mean,3.398849e+04,5.172364e+04,3.376945e+04,1.316976e+04,7.198633e+03,238.140855,3.122278e+08,6.900425e+04
std,2.294086e+05,2.402629e+05,1.561113e+05,6.575275e+04,5.966682e+04,2075.050043,1.282880e+09,2.761107e+05
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.000000e+00
25%,3.100000e+01,2.000000e+00,0.000000e+00,0.000000e+00,8.000000e+00,0.000000,7.610000e+02,1.400000e+01
50%,4.150000e+02,1.500000e+01,0.000000e+00,0.000000e+00,2.100000e+01,0.000000,6.096000e+03,8.400000e+01
75%,4.721000e+03,1.770000e+03,1.055000e+03,1.720000e+02,2.550000e+02,2.000000,1.506048e+07,7.833000e+03
max,1.799902e+07,1.660005e+07,1.659795e+07,3.291268e+06,5.619955e+06,221969.000000,2.311582e+10,1.660012e+07



🟢 QueryStartReport Table Details:
----------------------------------------
Shape: (24635, 11)
Memory usage: 183.93 MB

📋 Column Details:
    1. GatewayObjectId      | object       | Nulls:   0
    2. RequestId            | object       | Nulls:   0
    3. DataSource           | object       | Nulls:   0
    4. QueryTrackingId      | object       | Nulls:   0
    5. QueryExecutionStartTimeUTC | object       | Nulls:   0
    6. QueryType            | object       | Nulls:   0
    7. QueryText            | object       | Nulls:   0
    8. EvaluationContext    | object       | Nulls: 303
    9. SourceFile           | object       | Nulls:   0
   10. GatewayFolder        | object       | Nulls:   0
   11. LoadTimestamp        | datetime64[us] | Nulls:   0

📊 Sample Data (first 3 rows):


,GatewayObjectId,RequestId,DataSource,QueryTrackingId,QueryExecutionStartTimeUTC,QueryType,QueryText,EvaluationContext,SourceFile,GatewayFolder,LoadTimestamp
0,418b9fbd-21fe-4de2-ba0d-35a5968cf820,1464586c-781c-4833-9384-01761153d564,"[""{\""kind\"":\""SQL\"",\""path\"":\""EDW-PRD;DataHub...",0f9d722b-54d1-4bf6-8e42-8d9dcbb6bc2a,2025-09-04T16:48:08.7371826Z,DirectQuery,RXhlY3V0aW5nIHF1ZXJ5ICh0aW1lb3V0PTIyNSkgIjxjY2...,"{""serviceTraceContexts"":[{""serviceName"":""Power...",QueryStartReport_MDC-AZRDTGW-P01_20250904T1148...,Gateway1,2025-09-24 13:04:20.679230
1,418b9fbd-21fe-4de2-ba0d-35a5968cf820,f56bf6c3-331b-4d70-89da-1b8955aa95a8,"[""{\""kind\"":\""SQL\"",\""path\"":\""EDW-PRD;DataHub...",26a22a06-08a1-4acf-8ea1-04f8addc1f36,2025-09-04T16:48:08.7421825Z,DirectQuery,RXhlY3V0aW5nIHF1ZXJ5ICh0aW1lb3V0PTIyNSkgIjxjY2...,"{""serviceTraceContexts"":[{""serviceName"":""Power...",QueryStartReport_MDC-AZRDTGW-P01_20250904T1148...,Gateway1,2025-09-24 13:04:20.679230
2,418b9fbd-21fe-4de2-ba0d-35a5968cf820,0e5147f6-9aa6-4962-bf12-347ba9544033,"[""{\""kind\"":\""SQL\"",\""path\"":\""EDW-PRD;DataHub...",42b1f32b-2996-44c7-bf16-35abde4559a5,2025-09-04T16:48:08.7671862Z,DirectQuery,RXhlY3V0aW5nIHF1ZXJ5ICh0aW1lb3V0PTIyNSkgIjxjY2...,"{""serviceTraceContexts"":[{""serviceName"":""Power...",QueryStartReport_MDC-AZRDTGW-P01_20250904T1148...,Gateway1,2025-09-24 13:04:20.679230


In [40]:
# === Advanced Data Analysis and Relationships ===

print("🔗 ANALYZING DATA RELATIONSHIPS")
print("=" * 50)

# Check if both tables have data for relationship analysis
if not query_execution_table.empty and not query_start_table.empty:
    
    print("\n🔍 Looking for common columns for data joining...")
    
    # Find common columns (excluding metadata columns we added)
    execution_cols = set(query_execution_table.columns) - {'SourceFile', 'GatewayFolder', 'LoadTimestamp'}
    start_cols = set(query_start_table.columns) - {'SourceFile', 'GatewayFolder', 'LoadTimestamp'}
    common_cols = execution_cols.intersection(start_cols)
    
    print(f"📊 Common columns found: {len(common_cols)}")
    if common_cols:
        print(f"   🔗 Join candidates: {', '.join(sorted(common_cols))}")
        
        # Try to identify the best join column (usually RequestId)
        join_candidates = ['RequestId', 'request_id', 'RequestID', 'Id', 'id', 'ID']
        best_join_col = None
        
        for candidate in join_candidates:
            if candidate in common_cols:
                best_join_col = candidate
                break
        
        if best_join_col:
            print(f"   🎯 Recommended join column: {best_join_col}")
            
            # Analyze join potential
            exec_unique = query_execution_table[best_join_col].nunique()
            start_unique = query_start_table[best_join_col].nunique()
            exec_total = len(query_execution_table)
            start_total = len(query_start_table)
            
            print(f"\n📈 Join Analysis:")
            print(f"   QueryExecution - Unique {best_join_col}: {exec_unique:,} / Total: {exec_total:,}")
            print(f"   QueryStart - Unique {best_join_col}: {start_unique:,} / Total: {start_total:,}")
            
            # Find matching records
            exec_ids = set(query_execution_table[best_join_col].dropna())
            start_ids = set(query_start_table[best_join_col].dropna())
            matching_ids = exec_ids.intersection(start_ids)
            
            print(f"   🔄 Matching records: {len(matching_ids):,}")
            print(f"   📊 Match rate: {len(matching_ids)/max(len(exec_ids), len(start_ids))*100:.1f}%")
            
            if len(matching_ids) > 0:
                print(f"\n✅ Tables can be joined on '{best_join_col}' column")
                
                # Perform a sample join
                sample_join = query_execution_table.merge(
                    query_start_table, 
                    on=best_join_col, 
                    how='inner', 
                    suffixes=('_exec', '_start')
                )
                
                print(f"   🔗 Sample join result: {len(sample_join):,} records")
                print(f"   📋 Combined columns: {len(sample_join.columns)}")
                
                if len(sample_join) > 0:
                    print(f"\n📊 Sample joined data:")
                    display(sample_join.head(2))
            else:
                print(f"\n⚠️  No matching records found for joining")
        else:
            print(f"   ⚠️  No standard join column found in common columns")
    else:
        print("   ⚠️  No common columns found between tables")
        
    # Show unique values in key columns
    print(f"\n🏷️  DATA PROFILING:")
    
    for table_name, table_df in [("QueryExecution", query_execution_table), ("QueryStart", query_start_table)]:
        print(f"\n   {table_name} Table:")
        
        # Check for categorical columns with reasonable unique values
        for col in table_df.columns:
            if col not in ['SourceFile', 'GatewayFolder', 'LoadTimestamp']:
                unique_count = table_df[col].nunique()
                total_count = len(table_df)
                
                # Only show columns that might be useful for grouping (not too many unique values)
                if unique_count <= min(50, total_count * 0.5):
                    print(f"      {col}: {unique_count} unique values")
                    if unique_count <= 10:  # Show actual values for small sets
                        values = table_df[col].value_counts().head(5)
                        print(f"         Top values: {dict(values)}")

else:
    if query_execution_table.empty:
        print("❌ QueryExecutionReport table is empty - cannot perform relationship analysis")
    if query_start_table.empty:
        print("❌ QueryStartReport table is empty - cannot perform relationship analysis")

🔗 ANALYZING DATA RELATIONSHIPS

🔍 Looking for common columns for data joining...
📊 Common columns found: 5
   🔗 Join candidates: DataSource, GatewayObjectId, QueryTrackingId, QueryType, RequestId
   🎯 Recommended join column: RequestId

📈 Join Analysis:
   QueryExecution - Unique RequestId: 174,500 / Total: 451,619
   QueryStart - Unique RequestId: 7,939 / Total: 24,635
   🔄 Matching records: 7,771
   📊 Match rate: 4.5%

✅ Tables can be joined on 'RequestId' column
   🔗 Sample join result: 166,801 records
   📋 Combined columns: 30

📊 Sample joined data:


,GatewayObjectId_exec,RequestId,DataSource_exec,QueryTrackingId_exec,QueryExecutionEndTimeUTC,QueryExecutionDuration(ms),QueryType_exec,DataReadingAndSerializationDuration(ms),DataReadingDuration(ms),DataSerializationDuration(ms),...,GatewayObjectId_start,DataSource_start,QueryTrackingId_start,QueryExecutionStartTimeUTC,QueryType_start,QueryText,EvaluationContext,SourceFile_start,GatewayFolder_start,LoadTimestamp_start
0,418b9fbd-21fe-4de2-ba0d-35a5968cf820,b6d1edbf-674f-4b5d-9e03-b36e64b2d592,"[""{\""kind\"":\""SQL\"",\""path\"":\""EDW-PRD;EDW\""}""]",27ee5056-6b70-444c-a89e-de6f448780aa,2025-09-03T14:19:20.8551481Z,2235,DirectQuery,9.0,0.0,1.0,...,d58b530e-cec5-4620-a141-bfbc9c861992,"[""{\""kind\"":\""SQL\"",\""path\"":\""EDW-PRD;EDW\""}""]",9176f27c-8a34-4e8d-be0c-1e6fba3fe2bc,2025-09-03T14:19:20.9868051Z,DirectQuery,RXhlY3V0aW5nIHF1ZXJ5ICh0aW1lb3V0PTIyMykgIjxjY2...,"{""serviceTraceContexts"":[{""serviceName"":""Power...",QueryStartReport_MDC-AZRDTGW-P02_20250903T0919...,Gateway2,2025-09-24 13:04:21.614409
1,418b9fbd-21fe-4de2-ba0d-35a5968cf820,a0146f49-57ab-49e6-a6ca-b2b93ba17717,"[""{\""kind\"":\""SQL\"",\""path\"":\""EDW-PRD;EDW\""}""]",1ee855b1-186c-4b8e-80cb-e96c8e700c98,2025-09-03T14:19:46.3172731Z,477,DirectQuery,1.0,0.0,0.0,...,d58b530e-cec5-4620-a141-bfbc9c861992,"[""{\""kind\"":\""SQL\"",\""path\"":\""EDW-PRD;EDW\""}""]",b252239e-ce41-41d4-848e-1aa17bcc93cb,2025-09-03T14:19:45.1770571Z,DirectQuery,RXhlY3V0aW5nIHF1ZXJ5ICh0aW1lb3V0PTIyNSkgIjxjY2...,"{""serviceTraceContexts"":[{""serviceName"":""Power...",QueryStartReport_MDC-AZRDTGW-P02_20250903T0919...,Gateway2,2025-09-24 13:04:21.614409



🏷️  DATA PROFILING:

   QueryExecution Table:
      GatewayObjectId: 2 unique values
         Top values: {'d58b530e-cec5-4620-a141-bfbc9c861992': np.int64(301757), '418b9fbd-21fe-4de2-ba0d-35a5968cf820': np.int64(149862)}
      QueryType: 2 unique values
         Top values: {'Refresh': np.int64(344910), 'DirectQuery': np.int64(106709)}
      Success: 2 unique values
         Top values: {'Y': np.int64(438206), 'N': np.int64(13413)}

   QueryStart Table:
      GatewayObjectId: 2 unique values
         Top values: {'418b9fbd-21fe-4de2-ba0d-35a5968cf820': np.int64(12381), 'd58b530e-cec5-4620-a141-bfbc9c861992': np.int64(12254)}
      QueryType: 2 unique values
         Top values: {'Refresh': np.int64(18797), 'DirectQuery': np.int64(5838)}


In [41]:
# === JSON Column Detection and Flattening ===

def detect_and_flatten_json_columns(df, table_name):
    """
    Detect columns that contain JSON data and flatten them into separate columns.
    
    Args:
        df (pandas.DataFrame): The DataFrame to process
        table_name (str): Name of the table for logging purposes
    
    Returns:
        pandas.DataFrame: DataFrame with JSON columns flattened
    """
    print(f"\n🔍 Analyzing JSON columns in {table_name} table...")
    
    if df.empty:
        print(f"   ⚠️  {table_name} table is empty - skipping JSON flattening")
        return df
    
    json_columns = []
    df_flattened = df.copy()
    
    # Check each column for JSON content
    for col in df.columns:
        # Skip metadata columns we added
        if col in ['SourceFile', 'GatewayFolder', 'LoadTimestamp', 'source_file', 'folder']:
            continue
            
        print(f"\n   📋 Checking column: {col}")
        
        # Get a sample of non-null values to check
        non_null_values = df[col].dropna()
        
        if len(non_null_values) == 0:
            print(f"      ⚠️  Column '{col}' has no non-null values")
            continue
            
        # Take first few values to check for JSON
        sample_values = non_null_values.head(5).tolist()
        
        json_count = 0
        for value in sample_values:
            try:
                # Try to parse as JSON
                if isinstance(value, str) and (value.strip().startswith('{') or value.strip().startswith('[')):
                    json.loads(value)
                    json_count += 1
            except (json.JSONDecodeError, TypeError):
                continue
        
        # If majority of sample values are JSON, consider it a JSON column
        json_ratio = json_count / len(sample_values)
        
        if json_ratio >= 0.5:  # At least 50% of samples are JSON
            print(f"      ✅ JSON column detected ({json_count}/{len(sample_values)} samples are JSON)")
            json_columns.append(col)
            
            try:
                # Create a list to store flattened data
                flattened_rows = []
                
                for idx, row in df.iterrows():
                    row_dict = row.to_dict()
                    
                    if pd.notna(row[col]):
                        try:
                            # Parse JSON and flatten
                            json_data = json.loads(str(row[col]))
                            
                            if isinstance(json_data, dict):
                                # Flatten dictionary
                                for key, value in json_data.items():
                                    new_col_name = f"{col}_{key}"
                                    row_dict[new_col_name] = value
                            elif isinstance(json_data, list):
                                # Handle list - create indexed columns
                                for i, item in enumerate(json_data):
                                    new_col_name = f"{col}_{i}"
                                    if isinstance(item, dict):
                                        # Flatten each dict in the list
                                        for key, value in item.items():
                                            nested_col_name = f"{col}_{i}_{key}"
                                            row_dict[nested_col_name] = value
                                    else:
                                        row_dict[new_col_name] = item
                            
                            # Remove original JSON column
                            del row_dict[col]
                            
                        except (json.JSONDecodeError, TypeError) as e:
                            # If JSON parsing fails, keep original value
                            print(f"         ⚠️  JSON parse error for row {idx}: {str(e)[:50]}...")
                    else:
                        # Handle null values - remove original column
                        del row_dict[col]
                    
                    flattened_rows.append(row_dict)
                
                # Create new DataFrame with flattened data
                df_flattened = pd.DataFrame(flattened_rows)
                
                # Get new columns created from this JSON column
                original_cols = set(df.columns)
                new_cols = set(df_flattened.columns) - original_cols
                
                print(f"      📊 Created {len(new_cols)} new columns: {', '.join(sorted(list(new_cols))[:5])}{'...' if len(new_cols) > 5 else ''}")
                
            except Exception as e:
                print(f"      ❌ Error flattening JSON column '{col}': {str(e)}")
                # Keep original column if flattening fails
                continue
        else:
            print(f"      ➖ Not a JSON column ({json_count}/{len(sample_values)} samples are JSON)")
    
    if json_columns:
        print(f"\n✅ JSON flattening completed for {table_name}!")
        print(f"   📋 Processed JSON columns: {', '.join(json_columns)}")
        print(f"   📊 Original shape: {df.shape}")
        print(f"   📊 New shape: {df_flattened.shape}")
        print(f"   ➕ Columns added: {df_flattened.shape[1] - df.shape[1]}")
    else:
        print(f"\n➖ No JSON columns found in {table_name} table")
    
    return df_flattened

print("✅ JSON flattening functions defined successfully!")

✅ JSON flattening functions defined successfully!


In [42]:
# === Apply JSON Flattening to Both Tables ===

print("🚀 STARTING JSON COLUMN FLATTENING")
print("=" * 60)

# Flatten QueryExecutionReport table
print("🔵 Processing QueryExecutionReport table...")
query_execution_table_flattened = detect_and_flatten_json_columns(query_execution_table, "QueryExecutionReport")

print("\n" + "=" * 60)

# Flatten QueryStartReport table  
print("🟢 Processing QueryStartReport table...")
query_start_table_flattened = detect_and_flatten_json_columns(query_start_table, "QueryStartReport")

print("\n" + "=" * 60)
print("📊 JSON FLATTENING SUMMARY")
print("=" * 60)

# Compare before and after
print("🔵 QueryExecutionReport:")
print(f"   📊 Original: {query_execution_table.shape[0]:,} rows × {query_execution_table.shape[1]} columns")
print(f"   📊 Flattened: {query_execution_table_flattened.shape[0]:,} rows × {query_execution_table_flattened.shape[1]} columns")
print(f"   ➕ Columns added: {query_execution_table_flattened.shape[1] - query_execution_table.shape[1]}")

print("\n🟢 QueryStartReport:")
print(f"   📊 Original: {query_start_table.shape[0]:,} rows × {query_start_table.shape[1]} columns")  
print(f"   📊 Flattened: {query_start_table_flattened.shape[0]:,} rows × {query_start_table_flattened.shape[1]} columns")
print(f"   ➕ Columns added: {query_start_table_flattened.shape[1] - query_start_table.shape[1]}")

# Update the main table variables with flattened versions
query_execution_table = query_execution_table_flattened
query_start_table = query_start_table_flattened

print("\n✅ JSON flattening completed! Main table variables updated.")
print("   🔗 Use 'query_execution_table' and 'query_start_table' for further analysis")

# Show sample of new columns if any were added
if query_execution_table_flattened.shape[1] > 0:
    print(f"\n📋 QueryExecutionReport columns (showing first 10):")
    print(f"   {', '.join(query_execution_table.columns[:10])}")
    if query_execution_table.shape[1] > 10:
        print(f"   ... and {query_execution_table.shape[1] - 10} more columns")

if query_start_table_flattened.shape[1] > 0:
    print(f"\n📋 QueryStartReport columns (showing first 10):")
    print(f"   {', '.join(query_start_table.columns[:10])}")
    if query_start_table.shape[1] > 10:
        print(f"   ... and {query_start_table.shape[1] - 10} more columns")

🚀 STARTING JSON COLUMN FLATTENING
🔵 Processing QueryExecutionReport table...

🔍 Analyzing JSON columns in QueryExecutionReport table...

   📋 Checking column: GatewayObjectId
      ➖ Not a JSON column (0/5 samples are JSON)

   📋 Checking column: RequestId
      ➖ Not a JSON column (0/5 samples are JSON)

   📋 Checking column: DataSource
      ✅ JSON column detected (5/5 samples are JSON)
      📊 Created 5 new columns: DataSource_0, DataSource_1, DataSource_2, DataSource_3, DataSource_4

   📋 Checking column: QueryTrackingId
      ➖ Not a JSON column (0/5 samples are JSON)

   📋 Checking column: QueryExecutionEndTimeUTC
      ➖ Not a JSON column (0/5 samples are JSON)

   📋 Checking column: QueryExecutionDuration(ms)
      ➖ Not a JSON column (0/5 samples are JSON)

   📋 Checking column: QueryType
      ➖ Not a JSON column (0/5 samples are JSON)

   📋 Checking column: DataReadingAndSerializationDuration(ms)
      ➖ Not a JSON column (0/5 samples are JSON)

   📋 Checking column: DataRea

In [43]:
# === Save Processed Tables (Optional) ===

def save_tables_to_files(execution_df, start_df, output_dir="processed_data"):
    """
    Save the processed tables to CSV files for future use.
    """
    print(f"\n💾 SAVING PROCESSED TABLES")
    print("=" * 40)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    try:
        # Save QueryExecutionReport table
        if not execution_df.empty:
            execution_file = os.path.join(output_dir, f"query_execution_combined_{current_time}.csv")
            execution_df.to_csv(execution_file, index=False)
            file_size = os.path.getsize(execution_file)
            print(f"✅ QueryExecutionReport saved:")
            print(f"   📁 File: {execution_file}")
            print(f"   📊 Records: {len(execution_df):,}")
            print(f"   💾 Size: {file_size:,} bytes ({file_size/1024/1024:.2f} MB)")
        
        # Save QueryStartReport table
        if not start_df.empty:
            start_file = os.path.join(output_dir, f"query_start_combined_{current_time}.csv")
            start_df.to_csv(start_file, index=False)
            file_size = os.path.getsize(start_file)
            print(f"\n✅ QueryStartReport saved:")
            print(f"   📁 File: {start_file}")
            print(f"   📊 Records: {len(start_df):,}")
            print(f"   💾 Size: {file_size:,} bytes ({file_size/1024/1024:.2f} MB)")
        
        print(f"\n🎯 All processed tables saved to: {os.path.abspath(output_dir)}")
        
    except Exception as e:
        print(f"❌ Error saving tables: {str(e)}")

# Optionally save the tables
save_choice = input("\n💾 Do you want to save the processed tables to CSV files? (y/n): ").lower().strip()

if save_choice in ['y', 'yes', '1', 'true']:
    save_tables_to_files(query_execution_table, query_start_table)
else:
    print("⏭️  Skipping table save operation")

print(f"\n🎉 GATEWAY LOG ANALYSIS COMPLETE!")
print("=" * 50)
print(f"📊 Results Summary:")
print(f"   🔵 QueryExecutionReport: {len(query_execution_table):,} records loaded")
print(f"   🟢 QueryStartReport: {len(query_start_table):,} records loaded") 
print(f"   🔗 Tables available as: 'query_execution_table' and 'query_start_table'")
print(f"   ⏰ Processing completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if not query_execution_table.empty or not query_start_table.empty:
    print(f"\n💡 Next Steps:")
    print(f"   • Explore data: query_execution_table.head(), query_start_table.head()")
    print(f"   • Filter data: query_execution_table[query_execution_table['Status'] == 'Success']")
    print(f"   • Join tables: pd.merge(query_execution_table, query_start_table, on='RequestId')")
    print(f"   • Visualize: Use matplotlib/seaborn for charts and analysis")


💾 SAVING PROCESSED TABLES
✅ QueryExecutionReport saved:
   📁 File: processed_data\query_execution_combined_20250924_130525.csv
   📊 Records: 451,619
   💾 Size: 165,760,158 bytes (158.08 MB)

✅ QueryStartReport saved:
   📁 File: processed_data\query_start_combined_20250924_130525.csv
   📊 Records: 24,635
   💾 Size: 181,925,848 bytes (173.50 MB)

🎯 All processed tables saved to: c:\Users\aveekr\OneDrive - Microsoft\Documents\Demos\my_code_samples\azuresamples\pbigatewayloganalysis\processed_data

🎉 GATEWAY LOG ANALYSIS COMPLETE!
📊 Results Summary:
   🔵 QueryExecutionReport: 451,619 records loaded
   🟢 QueryStartReport: 24,635 records loaded
   🔗 Tables available as: 'query_execution_table' and 'query_start_table'
   ⏰ Processing completed at: 2025-09-24 13:05:38

💡 Next Steps:
   • Explore data: query_execution_table.head(), query_start_table.head()
   • Filter data: query_execution_table[query_execution_table['Status'] == 'Success']
   • Join tables: pd.merge(query_execution_table, que

In [44]:
# === TABLE SCHEMA INFORMATION ===

def display_table_schema(df, table_name):
    """
    Display comprehensive schema information for a DataFrame.
    """
    print(f"\n📋 {table_name.upper()} TABLE SCHEMA")
    print("=" * (len(table_name) + 20))
    
    if df.empty:
        print(f"❌ {table_name} table is empty")
        return
    
    print(f"📊 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
    
    print(f"\n📋 COLUMN DETAILS:")
    print("=" * 80)
    print(f"{'#':<3} {'Column Name':<25} {'Data Type':<15} {'Non-Null':<10} {'Nulls':<8} {'Sample Values'}")
    print("-" * 80)
    
    for i, col in enumerate(df.columns):
        dtype = str(df[col].dtype)
        non_null_count = df[col].count()
        null_count = df[col].isnull().sum()
        
      
        
        print(f"{i+1:<3} {col:<25} {dtype:<15} {non_null_count:<10,} {null_count:<8,} {sample_str}")
    
    # Data type summary
    dtype_counts = df.dtypes.value_counts()
    print(f"\n📊 DATA TYPE SUMMARY:")
    for dtype, count in dtype_counts.items():
        print(f"   {dtype}: {count} columns")
    
    # Missing values summary
    missing_cols = df.isnull().sum()
    missing_cols = missing_cols[missing_cols > 0].sort_values(ascending=False)
    
    if len(missing_cols) > 0:
        print(f"\n❗ COLUMNS WITH MISSING VALUES:")
        for col, missing_count in missing_cols.head(10).items():
            missing_pct = (missing_count / len(df)) * 100
            print(f"   {col}: {missing_count:,} ({missing_pct:.1f}%)")
        
        if len(missing_cols) > 10:
            print(f"   ... and {len(missing_cols) - 10} more columns with missing values")
    else:
        print(f"\n✅ NO MISSING VALUES FOUND")
    
    # Numeric columns summary
    numeric_cols = df.select_dtypes(include=['number']).columns
    if len(numeric_cols) > 0:
        print(f"\n📈 NUMERIC COLUMNS ({len(numeric_cols)} columns):")
        for col in numeric_cols:
            min_val = df[col].min()
            max_val = df[col].max()
            mean_val = df[col].mean()
            print(f"   {col}: Min={min_val}, Max={max_val}, Mean={mean_val:.2f}")
    
    # Categorical columns with low cardinality
    categorical_info = []
    for col in df.select_dtypes(include=['object']).columns:
        unique_count = df[col].nunique()
        if unique_count <= min(20, len(df) * 0.1):  # Low cardinality columns
            categorical_info.append((col, unique_count))
    
    if categorical_info:
        print(f"\n🏷️  CATEGORICAL COLUMNS (≤20 unique values):")
        for col, unique_count in categorical_info:
            top_values = df[col].value_counts().head(3)
            values_str = ', '.join([f"'{k}' ({v})" for k, v in top_values.items()])
            print(f"   {col}: {unique_count} unique - {values_str}")

print("🚀 Displaying schema information for both tables...")

🚀 Displaying schema information for both tables...


In [45]:
# === Optimized extraction for traceIds structure ===

def extract_ids_from_trace_structure(df, table_name):
    """
    Extract WorkspaceId and DatasetId from the specific traceIds structure in EvaluationContext.
    
    Expected structure:
    [{'serviceName': 'Power BI Datasets', 'traceIds': [
        {'key': 'DatasetId', 'value': 'c097b83d-16a9-4fc1-afee-be00914926ae'}, 
        {'key': 'WorkspaceId', 'value': 'A65A34D1-846F-42F4-BC7A-1654C7926D9B'}, 
        ...
    ]}]
    """
    print(f"\n🔍 Extracting IDs from traceIds structure in {table_name}...")
    print(f"   📋 Using column: {EVALUATION_CONTEXT_COLUMN}")
    
    if df.empty:
        print(f"   ⚠️  {table_name} table is empty")
        return df
    
    # Check if the configured EvaluationContext column exists
    if EVALUATION_CONTEXT_COLUMN not in df.columns:
        print(f"   ❌ Column '{EVALUATION_CONTEXT_COLUMN}' not found in {table_name}")
        print(f"   📋 Available columns: {', '.join(df.columns)}")
        return df
    
    print(f"   ✅ Found column with {df[EVALUATION_CONTEXT_COLUMN].count():,} non-null values")
    
    # Create a copy to work with
    df_enhanced = df.copy()
    
    # Initialize new columns
    df_enhanced['WorkspaceId'] = None
    df_enhanced['DatasetId'] = None
    df_enhanced['RootActivityId'] = None
    df_enhanced['CurrentActivityId'] = None
    df_enhanced['QueryType_TraceIds'] = None
    df_enhanced['SKU'] = None
    df_enhanced['ApplicationContext'] = None
    
    processed_count = 0
    error_count = 0
    workspace_extracted = 0
    dataset_extracted = 0
    
    # Process each row
    for idx, row in df.iterrows():
        if pd.notna(row[EVALUATION_CONTEXT_COLUMN]):
            try:
                # Parse the EvaluationContext
                eval_context = row[EVALUATION_CONTEXT_COLUMN]
                
                # Handle different input types
                if isinstance(eval_context, str):
                    # Parse JSON string
                    context_data = json.loads(eval_context)
                elif isinstance(eval_context, (list, dict)):
                    # Already parsed
                    context_data = eval_context
                else:
                    print(f"   ⚠️  Row {idx}: Unexpected data type {type(eval_context)}")
                    continue
                
                # Process the structure - should be a list
                if isinstance(context_data, list):
                    for service_item in context_data:
                        if isinstance(service_item, dict) and 'traceIds' in service_item:
                            trace_ids = service_item.get('traceIds', [])
                            
                            # Extract all key-value pairs from traceIds
                            for trace_item in trace_ids:
                                if isinstance(trace_item, dict) and 'key' in trace_item and 'value' in trace_item:
                                    key = trace_item['key']
                                    value = trace_item['value']
                                    
                                    # Map to appropriate columns
                                    if key == 'WorkspaceId':
                                        df_enhanced.at[idx, 'WorkspaceId'] = value
                                        workspace_extracted += 1
                                    elif key == 'DatasetId':
                                        df_enhanced.at[idx, 'DatasetId'] = value
                                        dataset_extracted += 1
                                    elif key == 'RootActivityId':
                                        df_enhanced.at[idx, 'RootActivityId'] = value
                                    elif key == 'CurrentActivityId':
                                        df_enhanced.at[idx, 'CurrentActivityId'] = value
                                    elif key == 'QueryType':
                                        df_enhanced.at[idx, 'QueryType_TraceIds'] = value
                                    elif key == 'SKU':
                                        df_enhanced.at[idx, 'SKU'] = value
                                    elif key == 'ApplicationContext':
                                        df_enhanced.at[idx, 'ApplicationContext'] = value
                
                processed_count += 1
                
                # Show progress for first few extractions
                if processed_count <= 3:
                    workspace_id = df_enhanced.at[idx, 'WorkspaceId']
                    dataset_id = df_enhanced.at[idx, 'DatasetId']
                    print(f"   📝 Row {idx}: WorkspaceId='{workspace_id}', DatasetId='{dataset_id}'")
                
            except Exception as e:
                error_count += 1
                if error_count <= 3:  # Only show first few errors
                    print(f"   ⚠️  Row {idx}: Error parsing - {str(e)[:100]}")
                continue
    
    # Summary
    print(f"\n✅ TraceIds extraction completed for {table_name}!")
    print(f"   📊 Processed rows: {processed_count:,}")
    print(f"   ❌ Error rows: {error_count:,}")
    print(f"   🆔 WorkspaceId extracted: {workspace_extracted:,} rows")
    print(f"   🔗 DatasetId extracted: {dataset_extracted:,} rows")
    
    # Show unique values
    unique_workspaces = df_enhanced['WorkspaceId'].nunique()
    unique_datasets = df_enhanced['DatasetId'].nunique()
    
    print(f"   📈 Unique WorkspaceIds: {unique_workspaces}")
    print(f"   📈 Unique DatasetIds: {unique_datasets}")
    
    if unique_workspaces > 0:
        top_workspaces = df_enhanced['WorkspaceId'].value_counts().head(3)
        print(f"   🏷️  Top WorkspaceIds: {dict(top_workspaces)}")
    
    if unique_datasets > 0:
        top_datasets = df_enhanced['DatasetId'].value_counts().head(3)
        print(f"   🏷️  Top DatasetIds: {dict(top_datasets)}")
    
    # Show additional extracted fields
    other_fields = ['RootActivityId', 'CurrentActivityId', 'QueryType_TraceIds', 'SKU']
    for field in other_fields:
        non_null_count = df_enhanced[field].notna().sum()
        if non_null_count > 0:
            print(f"   📋 {field}: {non_null_count:,} values extracted")
    
    return df_enhanced

print("✅ TraceIds extraction function defined!")

✅ TraceIds extraction function defined!


## 🔍 Extract WorkspaceId and DatasetId from TraceIds Structure

The EvaluationContext contains a specific traceIds structure with key-value pairs:
```json
[{'serviceName': 'Power BI Datasets', 'traceIds': [
    {'key': 'DatasetId', 'value': 'c097b83d-16a9-4fc1-afee-be00914926ae'}, 
    {'key': 'WorkspaceId', 'value': 'A65A34D1-846F-42F4-BC7A-1654C7926D9B'}, 
    {'key': 'RootActivityId', 'value': '...'},
    {'key': 'CurrentActivityId', 'value': '...'},
    {'key': 'QueryType', 'value': 'DirectQuery'},
    {'key': 'SKU', 'value': 'Premium'}
]}]
```

This specialized extraction function efficiently parses this structure to extract:
- **WorkspaceId** and **DatasetId** (primary identifiers)
- Additional metadata like RootActivityId, CurrentActivityId, QueryType, SKU

In [46]:
# === Apply TraceIds Extraction ===

print("🚀 APPLYING TRACEIDS EXTRACTION")
print("=" * 60)

# Apply the optimized traceIds extraction to Query Start table
query_start_table_trace_enhanced = extract_ids_from_trace_structure(query_start_table, "Query Start Report")

# Update the main table variable
query_start_table = query_start_table_trace_enhanced

# Display sample results
if not query_start_table.empty:
    trace_columns = [col for col in query_start_table.columns if col.endswith('_TraceIds') or col in ['WorkspaceId', 'DatasetId', 'RootActivityId', 'CurrentActivityId', 'SKU']]
    
    if trace_columns:
        print(f"\n📋 Sample of extracted TraceIds data:")
        sample_data = query_start_table[trace_columns].head(3)
        display(sample_data)
        
        # Show non-null counts for each column
        print(f"\n📊 Extraction Summary:")
        for col in trace_columns:
            non_null = query_start_table[col].notna().sum()
            if non_null > 0:
                print(f"   {col}: {non_null:,} values")

print(f"\n✅ TraceIds extraction completed! Updated table shape: {query_start_table.shape}")

🚀 APPLYING TRACEIDS EXTRACTION

🔍 Extracting IDs from traceIds structure in Query Start Report...
   📋 Using column: EvaluationContext_serviceTraceContexts
   ✅ Found column with 24,332 non-null values
   📝 Row 0: WorkspaceId='A65A34D1-846F-42F4-BC7A-1654C7926D9B', DatasetId='c097b83d-16a9-4fc1-afee-be00914926ae'
   📝 Row 1: WorkspaceId='A65A34D1-846F-42F4-BC7A-1654C7926D9B', DatasetId='c097b83d-16a9-4fc1-afee-be00914926ae'
   📝 Row 2: WorkspaceId='A65A34D1-846F-42F4-BC7A-1654C7926D9B', DatasetId='c097b83d-16a9-4fc1-afee-be00914926ae'

✅ TraceIds extraction completed for Query Start Report!
   📊 Processed rows: 24,332
   ❌ Error rows: 0
   🆔 WorkspaceId extracted: 23,854 rows
   🔗 DatasetId extracted: 23,854 rows
   📈 Unique WorkspaceIds: 34
   📈 Unique DatasetIds: 400
   🏷️  Top WorkspaceIds: {'9E4785A2-9CD3-4001-999A-2E5352D12F80': np.int64(5749), 'A65A34D1-846F-42F4-BC7A-1654C7926D9B': np.int64(5160), 'E00ED1BD-2A20-4504-96E6-AB4FBC20FF6C': np.int64(3070)}
   🏷️  Top DatasetIds: {'c

,WorkspaceId,DatasetId,RootActivityId,CurrentActivityId,QueryType_TraceIds,SKU
0,A65A34D1-846F-42F4-BC7A-1654C7926D9B,c097b83d-16a9-4fc1-afee-be00914926ae,1464586c-781c-4833-9384-01761153d564,a8a133a8-467c-428a-abda-1eecfef7ad50,DirectQuery,Premium
1,A65A34D1-846F-42F4-BC7A-1654C7926D9B,c097b83d-16a9-4fc1-afee-be00914926ae,f56bf6c3-331b-4d70-89da-1b8955aa95a8,9946f706-22bc-4f76-80ac-3e1c0fb64dc6,DirectQuery,Premium
2,A65A34D1-846F-42F4-BC7A-1654C7926D9B,c097b83d-16a9-4fc1-afee-be00914926ae,0e5147f6-9aa6-4962-bf12-347ba9544033,e3003014-9493-41ca-b2fd-6561113e59dc,DirectQuery,Premium



📊 Extraction Summary:
   WorkspaceId: 23,854 values
   DatasetId: 23,854 values
   RootActivityId: 24,320 values
   CurrentActivityId: 24,320 values
   QueryType_TraceIds: 23,854 values
   SKU: 23,854 values

✅ TraceIds extraction completed! Updated table shape: (24635, 18)


In [47]:
# === Parse ApplicationContext JSON ===

def parse_application_context(df, table_name):
    """
    Parse the ApplicationContext JSON field to extract additional information like ReportId, VisualId, etc.
    
    Expected ApplicationContext JSON structure:
    {
        "DatasetId": "c097b83d-16a9-4fc1-afee-be00914926ae",
        "Sources": [
            {
                "ReportId": "23f4bdec-7285-43a9-a7a5-916860bcbd2a",
                "VisualId": "eb39794303e2d9aa39ab",
                "HostProperties": {
                    "ConsumptionMethod": "Power BI Web App",
                    "UserSession": "a0ba8b5e-c7f0-4bc6-ba0a-8cc048bd3bd9"
                }
            }
        ]
    }
    """
    print(f"\n🔍 Parsing ApplicationContext JSON in {table_name}...")
    
    if df.empty or 'ApplicationContext' not in df.columns:
        print(f"   ⚠️  ApplicationContext column not found or table is empty")
        return df
    
    # Create a copy to work with
    df_enhanced = df.copy()
    
    # Initialize new columns for ApplicationContext fields
    df_enhanced['AppContext_DatasetId'] = None
    df_enhanced['ReportId'] = None
    df_enhanced['VisualId'] = None
    df_enhanced['ConsumptionMethod'] = None
    df_enhanced['UserSession'] = None
    
    processed_count = 0
    error_count = 0
    
    # Process each row that has ApplicationContext data
    for idx, row in df.iterrows():
        if pd.notna(row['ApplicationContext']):
            try:
                # Parse the ApplicationContext JSON
                app_context = json.loads(row['ApplicationContext'])
                
                # Extract DatasetId from ApplicationContext
                if 'DatasetId' in app_context:
                    df_enhanced.at[idx, 'AppContext_DatasetId'] = app_context['DatasetId']
                
                # Extract information from Sources array
                if 'Sources' in app_context and isinstance(app_context['Sources'], list):
                    for source in app_context['Sources']:
                        if isinstance(source, dict):
                            # Extract ReportId and VisualId
                            if 'ReportId' in source:
                                df_enhanced.at[idx, 'ReportId'] = source['ReportId']
                            if 'VisualId' in source:
                                df_enhanced.at[idx, 'VisualId'] = source['VisualId']
                            
                            # Extract HostProperties
                            if 'HostProperties' in source and isinstance(source['HostProperties'], dict):
                                host_props = source['HostProperties']
                                if 'ConsumptionMethod' in host_props:
                                    df_enhanced.at[idx, 'ConsumptionMethod'] = host_props['ConsumptionMethod']
                                if 'UserSession' in host_props:
                                    df_enhanced.at[idx, 'UserSession'] = host_props['UserSession']
                            
                            # Usually only process first source for simplicity
                            break
                
                processed_count += 1
                
                # Show progress for first few extractions
                if processed_count <= 3:
                    report_id = df_enhanced.at[idx, 'ReportId']
                    consumption = df_enhanced.at[idx, 'ConsumptionMethod']
                    print(f"   📝 Row {idx}: ReportId='{report_id}', Method='{consumption}'")
                
            except Exception as e:
                error_count += 1
                if error_count <= 3:
                    print(f"   ⚠️  Row {idx}: Error parsing ApplicationContext - {str(e)[:100]}")
                continue
    
    # Summary
    app_context_fields = ['AppContext_DatasetId', 'ReportId', 'VisualId', 'ConsumptionMethod', 'UserSession']
    
    print(f"\n✅ ApplicationContext parsing completed!")
    print(f"   📊 Processed rows: {processed_count:,}")
    print(f"   ❌ Error rows: {error_count:,}")
    
    for field in app_context_fields:
        non_null_count = df_enhanced[field].notna().sum()
        unique_count = df_enhanced[field].nunique()
        if non_null_count > 0:
            print(f"   📋 {field}: {non_null_count:,} values ({unique_count} unique)")
    
    return df_enhanced

# Apply ApplicationContext parsing if the column exists
if 'ApplicationContext' in query_start_table.columns:
    print("🚀 PARSING APPLICATIONCONTEXT")
    print("=" * 50)
    
    query_start_table = parse_application_context(query_start_table, "Query Start Report")
    
    print(f"\n✅ ApplicationContext parsing completed! Updated table shape: {query_start_table.shape}")
else:
    print("📋 ApplicationContext column not found in current table")

print("✅ ApplicationContext parsing function defined!")

🚀 PARSING APPLICATIONCONTEXT

🔍 Parsing ApplicationContext JSON in Query Start Report...
   📝 Row 0: ReportId='23f4bdec-7285-43a9-a7a5-916860bcbd2a', Method='Power BI Web App'
   📝 Row 1: ReportId='23f4bdec-7285-43a9-a7a5-916860bcbd2a', Method='Power BI Web App'
   📝 Row 2: ReportId='23f4bdec-7285-43a9-a7a5-916860bcbd2a', Method='Power BI Web App'

✅ ApplicationContext parsing completed!
   📊 Processed rows: 5,474
   ❌ Error rows: 0
   📋 AppContext_DatasetId: 5,474 values (32 unique)
   📋 ReportId: 4,665 values (33 unique)
   📋 VisualId: 4,597 values (99 unique)
   📋 ConsumptionMethod: 3,546 values (3 unique)
   📋 UserSession: 3,546 values (226 unique)

✅ ApplicationContext parsing completed! Updated table shape: (24635, 23)
✅ ApplicationContext parsing function defined!


In [ ]:
# === Comprehensive TraceIds Data Summary ===

print("📊 COMPREHENSIVE TRACEIDS EXTRACTION SUMMARY")
print("=" * 70)

# Define all the columns we extracted from traceIds and ApplicationContext
trace_columns = [
    'WorkspaceId', 'DatasetId', 'RootActivityId', 'CurrentActivityId', 
    'QueryType_TraceIds', 'SKU', 'AppContext_DatasetId', 'ReportId', 'VisualId', 
    'ConsumptionMethod', 'UserSession'
]

# Check which columns actually exist in our table
existing_trace_columns = [col for col in trace_columns if col in query_start_table.columns]

if existing_trace_columns:
    print(f"📋 Extracted TraceIds and ApplicationContext fields:")
    print("-" * 50)
    
    for col in existing_trace_columns:
        non_null_count = query_start_table[col].notna().sum()
        unique_count = query_start_table[col].nunique()
        total_rows = len(query_start_table)
        coverage_pct = (non_null_count / total_rows) * 100 if total_rows > 0 else 0
        
        print(f"   {col:<25} : {non_null_count:>6,} values ({unique_count:>4} unique) - {coverage_pct:5.1f}% coverage")
    
    # Show sample data with the most important columns
    key_columns = ['RequestId'] + [col for col in ['WorkspaceId', 'DatasetId', 'ReportId', 'ConsumptionMethod'] if col in query_start_table.columns]
    
    print(f"\n📋 Sample of key extracted data:")
    sample_data = query_start_table[key_columns].dropna(subset=[col for col in key_columns[1:] if col in query_start_table.columns], how='all').head(5)
    
    if not sample_data.empty:
        display(sample_data)
    else:
        print("   ⚠️  No rows with extracted traceIds data found")
    
    
    
    # Show distribution of key categorical fields
    categorical_fields = ['ConsumptionMethod', 'QueryType_TraceIds', 'SKU']
    for field in categorical_fields:
        if field in query_start_table.columns:
            value_counts = query_start_table[field].value_counts()
            if not value_counts.empty:
                print(f"\n📈 {field} distribution:")
                for value, count in value_counts.head(5).items():
                    print(f"   '{value}': {count:,} occurrences")
    
else:
    print("❌ No traceIds columns found in the table")
    print("💡 Make sure to run the traceIds extraction functions first")

print(f"\n✅ Current table shape: {query_start_table.shape}")
print(f"📋 Total columns: {len(query_start_table.columns)}")
display(query_start_table.head(3))



📊 COMPREHENSIVE TRACEIDS EXTRACTION SUMMARY
📋 Extracted TraceIds and ApplicationContext fields:
--------------------------------------------------
   WorkspaceId               : 23,854 values (  34 unique) -  96.8% coverage
   DatasetId                 : 23,854 values ( 400 unique) -  96.8% coverage
   RootActivityId            : 24,320 values (7630 unique) -  98.7% coverage
   CurrentActivityId         : 24,320 values (14754 unique) -  98.7% coverage
   QueryType_TraceIds        : 23,854 values (   2 unique) -  96.8% coverage
   SKU                       : 23,854 values (   2 unique) -  96.8% coverage
   AppContext_DatasetId      :  5,474 values (  32 unique) -  22.2% coverage
   ReportId                  :  4,665 values (  33 unique) -  18.9% coverage
   VisualId                  :  4,597 values (  99 unique) -  18.7% coverage
   ConsumptionMethod         :  3,546 values (   3 unique) -  14.4% coverage
   UserSession               :  3,546 values ( 226 unique) -  14.4% coverage

📋 Sa

,RequestId,WorkspaceId,DatasetId,ReportId,ConsumptionMethod
0,1464586c-781c-4833-9384-01761153d564,A65A34D1-846F-42F4-BC7A-1654C7926D9B,c097b83d-16a9-4fc1-afee-be00914926ae,23f4bdec-7285-43a9-a7a5-916860bcbd2a,Power BI Web App
1,f56bf6c3-331b-4d70-89da-1b8955aa95a8,A65A34D1-846F-42F4-BC7A-1654C7926D9B,c097b83d-16a9-4fc1-afee-be00914926ae,23f4bdec-7285-43a9-a7a5-916860bcbd2a,Power BI Web App
2,0e5147f6-9aa6-4962-bf12-347ba9544033,A65A34D1-846F-42F4-BC7A-1654C7926D9B,c097b83d-16a9-4fc1-afee-be00914926ae,23f4bdec-7285-43a9-a7a5-916860bcbd2a,Power BI Web App
3,f5c3fca4-90f5-437d-8008-b046d2818d5a,A65A34D1-846F-42F4-BC7A-1654C7926D9B,c097b83d-16a9-4fc1-afee-be00914926ae,23f4bdec-7285-43a9-a7a5-916860bcbd2a,Power BI Web App
4,f5c3fca4-90f5-437d-8008-b046d2818d5a,A65A34D1-846F-42F4-BC7A-1654C7926D9B,c097b83d-16a9-4fc1-afee-be00914926ae,23f4bdec-7285-43a9-a7a5-916860bcbd2a,Power BI Web App



📈 ConsumptionMethod distribution:
   'Power BI Web App': 3,500 occurrences
   'Power BI Teams Personal App': 42 occurrences
   'Power BI Office Personal App': 4 occurrences

📈 QueryType_TraceIds distribution:
   'Import': 18,331 occurrences
   'DirectQuery': 5,523 occurrences

📈 SKU distribution:
   'Premium': 23,599 occurrences
   'Pro': 255 occurrences

✅ Current table shape: (24635, 23)
📋 Total columns: 23


,GatewayObjectId,RequestId,DataSource,QueryTrackingId,QueryExecutionStartTimeUTC,QueryType,QueryText,SourceFile,GatewayFolder,LoadTimestamp,...,RootActivityId,CurrentActivityId,QueryType_TraceIds,SKU,ApplicationContext,AppContext_DatasetId,ReportId,VisualId,ConsumptionMethod,UserSession
0,418b9fbd-21fe-4de2-ba0d-35a5968cf820,1464586c-781c-4833-9384-01761153d564,"[""{\""kind\"":\""SQL\"",\""path\"":\""EDW-PRD;DataHub...",0f9d722b-54d1-4bf6-8e42-8d9dcbb6bc2a,2025-09-04T16:48:08.7371826Z,DirectQuery,RXhlY3V0aW5nIHF1ZXJ5ICh0aW1lb3V0PTIyNSkgIjxjY2...,QueryStartReport_MDC-AZRDTGW-P01_20250904T1148...,Gateway1,2025-09-24 13:04:20.679230,...,1464586c-781c-4833-9384-01761153d564,a8a133a8-467c-428a-abda-1eecfef7ad50,DirectQuery,Premium,"{""DatasetId"":""c097b83d-16a9-4fc1-afee-be009149...",c097b83d-16a9-4fc1-afee-be00914926ae,23f4bdec-7285-43a9-a7a5-916860bcbd2a,eb39794303e2d9aa39ab,Power BI Web App,a0ba8b5e-c7f0-4bc6-ba0a-8cc048bd3bd9
1,418b9fbd-21fe-4de2-ba0d-35a5968cf820,f56bf6c3-331b-4d70-89da-1b8955aa95a8,"[""{\""kind\"":\""SQL\"",\""path\"":\""EDW-PRD;DataHub...",26a22a06-08a1-4acf-8ea1-04f8addc1f36,2025-09-04T16:48:08.7421825Z,DirectQuery,RXhlY3V0aW5nIHF1ZXJ5ICh0aW1lb3V0PTIyNSkgIjxjY2...,QueryStartReport_MDC-AZRDTGW-P01_20250904T1148...,Gateway1,2025-09-24 13:04:20.679230,...,f56bf6c3-331b-4d70-89da-1b8955aa95a8,9946f706-22bc-4f76-80ac-3e1c0fb64dc6,DirectQuery,Premium,"{""DatasetId"":""c097b83d-16a9-4fc1-afee-be009149...",c097b83d-16a9-4fc1-afee-be00914926ae,23f4bdec-7285-43a9-a7a5-916860bcbd2a,ef960bf1e53651534481,Power BI Web App,a0ba8b5e-c7f0-4bc6-ba0a-8cc048bd3bd9
2,418b9fbd-21fe-4de2-ba0d-35a5968cf820,0e5147f6-9aa6-4962-bf12-347ba9544033,"[""{\""kind\"":\""SQL\"",\""path\"":\""EDW-PRD;DataHub...",42b1f32b-2996-44c7-bf16-35abde4559a5,2025-09-04T16:48:08.7671862Z,DirectQuery,RXhlY3V0aW5nIHF1ZXJ5ICh0aW1lb3V0PTIyNSkgIjxjY2...,QueryStartReport_MDC-AZRDTGW-P01_20250904T1148...,Gateway1,2025-09-24 13:04:20.679230,...,0e5147f6-9aa6-4962-bf12-347ba9544033,e3003014-9493-41ca-b2fd-6561113e59dc,DirectQuery,Premium,"{""DatasetId"":""c097b83d-16a9-4fc1-afee-be009149...",c097b83d-16a9-4fc1-afee-be00914926ae,23f4bdec-7285-43a9-a7a5-916860bcbd2a,c9bd1d33ba33d320e8c3,Power BI Web App,a0ba8b5e-c7f0-4bc6-ba0a-8cc048bd3bd9


In [ ]:
# === Save Enhanced Query Start Report ===

import os
from datetime import datetime

def save_enhanced_query_start_report(df, base_path):
    """
    Save the enhanced query start report with all extracted columns to a separate folder.
    
    Args:
        df (pandas.DataFrame): The enhanced query start DataFrame
        base_path (str): Base path where to create the output folder
    
    Returns:
        str: Path to the saved file
    """
    print(f"💾 SAVING ENHANCED QUERY START REPORT")
    print("=" * 50)
    
    # Create timestamp for unique folder/file naming
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Create enhanced_reports folder
    enhanced_reports_dir = os.path.join(base_path, "enhanced_reports")
    os.makedirs(enhanced_reports_dir, exist_ok=True)
    
    print(f"📁 Output directory: {enhanced_reports_dir}")
    
    # Remove QueryText and EvaluationContext columns to reduce file size
    columns_to_remove = ['QueryText', EVALUATION_CONTEXT_COLUMN]
    existing_columns_to_remove = [col for col in columns_to_remove if col in df.columns]
    
    if existing_columns_to_remove:
        print(f"\n🗑️  Removing columns to optimize file size:")
        for col in existing_columns_to_remove:
            print(f"   ❌ Removing: {col}")
        df_clean = df.drop(columns=existing_columns_to_remove)
    else:
        df_clean = df.copy()
    
    # Display current data summary
    print(f"\n📊 Data Summary (after column removal):")
    print(f"   📋 Total rows: {len(df_clean):,}")
    print(f"   📋 Total columns: {len(df_clean.columns)} (removed {len(existing_columns_to_remove)} columns)")
    
    # Show extracted columns summary
    extracted_columns = ['WorkspaceId', 'DatasetId', 'RootActivityId', 'CurrentActivityId', 'QueryType_TraceIds', 'SKU']
    available_extracted = [col for col in extracted_columns if col in df_clean.columns]
    
    if available_extracted:
        print(f"\n✅ Extracted Columns Available:")
        for col in available_extracted:
            non_null_count = df_clean[col].notna().sum()
            unique_count = df_clean[col].nunique()
            coverage_pct = (non_null_count / len(df_clean)) * 100
            print(f"   📈 {col}: {non_null_count:,} values ({coverage_pct:.1f}% coverage, {unique_count} unique)")
    
    # Define output filename
    output_filename = f"query_start_enhanced_{timestamp}.csv"
    output_path = os.path.join(enhanced_reports_dir, output_filename)
    
    # Save to CSV (using the cleaned dataframe)
    print(f"\n💾 Saving to: {output_filename}")
    df_clean.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    # Verify the saved file
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
        print(f"✅ File saved successfully!")
        print(f"   📊 File size: {file_size:.2f} MB")
        print(f"   📄 Full path: {output_path}")
        
        # Create a summary file with column information
        summary_filename = f"query_start_enhanced_summary_{timestamp}.txt"
        summary_path = os.path.join(enhanced_reports_dir, summary_filename)
        
        with open(summary_path, 'w', encoding='utf-8') as f:
            f.write(f"Enhanced Query Start Report Summary\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"=" * 50 + "\n\n")
            
            f.write(f"Dataset Information:\n")
            f.write(f"- Total Rows: {len(df_clean):,}\n")
            f.write(f"- Total Columns: {len(df_clean.columns)} (after removing {len(existing_columns_to_remove)} columns)\n")
            f.write(f"- File Size: {file_size:.2f} MB\n")
            if existing_columns_to_remove:
                f.write(f"- Removed Columns: {', '.join(existing_columns_to_remove)}\n")
            f.write(f"\n")
            
            f.write(f"Column Summary:\n")
            f.write(f"- All Columns ({len(df_clean.columns)}): {', '.join(df_clean.columns)}\n\n")
            
            if available_extracted:
                f.write(f"Extracted Enhancement Columns:\n")
                for col in available_extracted:
                    non_null_count = df_clean[col].notna().sum()
                    unique_count = df_clean[col].nunique()
                    coverage_pct = (non_null_count / len(df_clean)) * 100
                    f.write(f"- {col}: {non_null_count:,} values ({coverage_pct:.1f}% coverage, {unique_count} unique)\n")
                
                f.write(f"\nTop Values for Key Columns:\n")
                for col in ['WorkspaceId', 'DatasetId']:
                    if col in df_clean.columns and df_clean[col].notna().sum() > 0:
                        top_values = df_clean[col].value_counts().head(5)
                        f.write(f"\n{col} (Top 5):\n")
                        for val, count in top_values.items():
                            f.write(f"  {val}: {count:,} occurrences\n")
        
        print(f"   📄 Summary saved: {summary_filename}")
        
        return output_path
    else:
        print(f"❌ Error: Failed to save file")
        return None

print("✅ Save function defined!")

In [ ]:
# === Execute Save Operation ===

print("🚀 EXECUTING SAVE OPERATION")
print("=" * 60)

# Check current working directory and data
current_path = os.getcwd()
print(f"📁 Current working directory: {current_path}")

# Show current query_start_table info
if 'query_start_table' in locals():
    print(f"\n📊 Current query_start_table status:")
    print(f"   📋 Shape: {query_start_table.shape}")
    print(f"   📋 Columns: {list(query_start_table.columns)}")
    
    # Check for enhanced columns
    enhanced_cols = ['WorkspaceId', 'DatasetId', 'RootActivityId', 'CurrentActivityId', 'QueryType_TraceIds', 'SKU']
    available_enhanced = [col for col in enhanced_cols if col in query_start_table.columns]
    
    if available_enhanced:
        print(f"   ✅ Enhanced columns found: {available_enhanced}")
        
        # Save the enhanced report
        saved_path = save_enhanced_query_start_report(query_start_table, current_path)
        
        if saved_path:
            print(f"\n🎉 SUCCESS! Enhanced Query Start Report saved successfully!")
            print(f"📁 Check the 'enhanced_reports' folder for your files")
        else:
            print(f"\n❌ Failed to save the report")
    else:
        print(f"   ⚠️  No enhanced columns found. Available columns: {list(query_start_table.columns)}")
else:
    print(f"❌ query_start_table not found in current variables")
    print(f"Available DataFrames: {[var for var in locals() if isinstance(locals()[var], pd.DataFrame)]}")